# 单向量搜索

一旦你插入了你的数据，下一步就是在Milvus中对你的收藏进行相似性搜索。

Milvus允许您进行两种类型的搜索，具体取决于集合中向量场的数量：

- 单向量搜索：如果你的集合只有一个向量字段，使用[`search()`](https://milvus.io/api-reference/pymilvus/v2.4.x/MilvusClient/Vector/search.md)方法来查找最相似的实体。此方法将您的查询向量与集合中的现有向量进行比较，并返回最接近匹配项的ID（沿着）以及它们之间的距离。可选地，它还可以返回结果的向量值和元数据。
- 多向量搜索：对于具有两个或更多向量字段的集合，请使用`hybrid_search()`方法。此方法执行多个近似最近邻（ANN）搜索请求，并在重新排序后组合结果以返回最相关的匹配。

本指南重点介绍如何在Milvus中执行单向量搜索。有关多向量搜索的详细信息，请参阅[混合搜索](https://milvus.io/docs/multi-vector-search.md)。


## 概述


有多种搜索类型可满足不同的要求：

- [基本搜索：包括单向量搜索、批量向量搜索、分区搜索和指定输出字段的搜索。](https://milvus.io/docs/single-vector-search.md#Basic-search)
- [过滤搜索](https://milvus.io/docs/single-vector-search.md#Filtered-search)：根据标量字段设置过滤条件，以优化搜索结果。
- [范围搜索](https://milvus.io/docs/single-vector-search.md#Range-search)：查找距离查询向量特定距离范围内的向量。
- [分类搜索：根据特定字段对搜索结果进行分组，以确保结果的多样性。](https://milvus.io/docs/single-vector-search.md#Grouping-search)

此页面上的代码片段使用新[的MilvusClient](https://milvus.io/api-reference/pymilvus/v2.4.x/About.md)（Python）与Milvus交互。其他语言的新MilvusClient SDK将在未来的更新中发布。


## 筹备工作


下面的代码片段重新使用现有代码来建立与Milvus的连接并快速设置集合。

In [4]:
import random
from pymilvus import MilvusClient

# 1. Set up a Milvus client
client = MilvusClient(
    uri="http://localhost:19530"
)

if client.has_collection("quick_setup"):
    client.drop_collection("quick_setup")

# 2. Create a collection
client.create_collection(
    collection_name="quick_setup",
    dimension=5,
    metric_type="IP"
)
    
# 2. Insert randomly generated vectors 
colors = ["green", "blue", "yellow", "red", "black", "white", "purple", "pink", "orange", "brown", "grey"]
data = [ {"id": i, "vector": [ random.uniform(-1, 1) for _ in range(5) ], "color": f"{random.choice(colors)}_{str(random.randint(1000, 9999))}" } for i in range(1000) ]

res = client.insert(
    collection_name="quick_setup",
    data=data
)

# print(res)

## 基本搜索

当发送`search`请求时，您可以提供一个或多个表示查询嵌入的向量值和一个指示返回结果数量的`limit`值。

根据您的数据和查询向量，您可能会得到少于`limit`的结果。当`limit`大于查询的可能匹配向量数时，就会发生这种情况。


### 单向量搜索


单向量搜索是Milvus中`search`操作的最简单形式，旨在查找与给定查询向量最相似的向量。

要执行单向量搜索，请指定目标集合名称、查询向量和所需的结果数（`limit`）。此操作返回一个结果集，其中包括最相似的向量、它们的ID以及与查询向量的距离。

以下是搜索与查询向量最相似的前5个实体的示例：

In [6]:
# Single vector search
res = client.search(
    collection_name="quick_setup", # Replace with the actual name of your collection
    # Replace with your query vector
    data=[[0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592]],
    limit=5, # Max. number of search results to return
    search_params={"metric_type": "IP", "params": {}} # Search parameters
)

# Convert the output to a formatted JSON string
result = json.dumps(res, indent=4)
print(result)

[
    [
        {
            "id": 572,
            "distance": 1.870904803276062,
            "entity": {}
        },
        {
            "id": 894,
            "distance": 1.842880129814148,
            "entity": {}
        },
        {
            "id": 751,
            "distance": 1.6808228492736816,
            "entity": {}
        },
        {
            "id": 673,
            "distance": 1.6371359825134277,
            "entity": {}
        },
        {
            "id": 364,
            "distance": 1.6252869367599487,
            "entity": {}
        }
    ]
]


输出类似于以下内容：


```python
[
    [
        {
            "id": 0,
            "distance": 1.4093276262283325,
            "entity": {}
        },
        {
            "id": 4,
            "distance": 0.9902134537696838,
            "entity": {}
        },
        {
            "id": 1,
            "distance": 0.8519943356513977,
            "entity": {}
        },
        {
            "id": 5,
            "distance": 0.7972343564033508,
            "entity": {}
        },
        {
            "id": 2,
            "distance": 0.5928734540939331,
            "entity": {}
        }
    ]
]
```

输出显示了与查询向量最近的前5个邻居，包括它们的唯一ID和计算的距离。

### 多向量搜索


批量向量搜索扩展了[单向量搜索的](https://milvus.io/docs/single-vector-search.md#Single-Vector-Search)概念，允许在单个请求中搜索多个查询向量。这种类型的搜索非常适合需要为一组查询向量找到相似向量的场景，从而显著减少所需的时间和计算资源。

在批量向量搜索中，您可以在`data`字段中包含多个查询向量。系统并行处理这些向量，为每个查询向量返回一个单独的结果集，每个集包含在集合中找到的最接近的匹配。

下面是一个从两个查询向量中搜索两组不同的最相似实体的例子：

In [8]:
# Bulk-vector search
res = client.search(
    collection_name="quick_setup", # Replace with the actual name of your collection
    data=[
        [0.19886812562848388, 0.06023560599112088, 0.6976963061752597, 0.2614474506242501, 0.838729485096104],
        [0.3172005263489739, 0.9719044792798428, -0.36981146090600725, -0.4860894583077995, 0.95791889146345]
    ], # Replace with your query vectors
    limit=2, # Max. number of search results to return
    search_params={"metric_type": "IP", "params": {}} # Search parameters
)

result = json.dumps(res, indent=4)
print(result)

[
    [
        {
            "id": 738,
            "distance": 1.678102731704712,
            "entity": {}
        },
        {
            "id": 48,
            "distance": 1.6663272380828857,
            "entity": {}
        }
    ],
    [
        {
            "id": 462,
            "distance": 2.5402042865753174,
            "entity": {}
        },
        {
            "id": 782,
            "distance": 2.4869089126586914,
            "entity": {}
        }
    ]
]



输出类似于以下内容：

```python
[
    [
        {
            "id": 1,
            "distance": 1.3017789125442505,
            "entity": {}
        },
        {
            "id": 7,
            "distance": 1.2419954538345337,
            "entity": {}
        }
    ], # Result set 1
    [
        {
            "id": 3,
            "distance": 2.3358664512634277,
            "entity": {}
        },
        {
            "id": 8,
            "distance": 0.5642921924591064,
            "entity": {}
        }
    ] # Result set 2
]
```

结果包括两组最近邻，每个查询向量一组，展示了批量向量搜索在一次处理多个查询向量时的效率。



### 分割搜索

分区搜索将搜索范围缩小到集合的特定子集或分区。这对于有组织的数据集特别有用，其中数据被分割为逻辑或分类分区，通过减少要扫描的数据量来实现更快的搜索操作。

要执行分区搜索，只需在搜索请求的`partition_names`中包含目标分区的名称。这指定`search`操作仅考虑指定分区内的向量。

以下是在`partition_1`中搜索实体的示例：

```python
# Search in partition_1
res = client.search(
    collection_name="test_collection", # Replace with the actual name of your collection
    data=[[0.02174828545444263, 0.058611125483182924, 0.6168633415965343, -0.7944160935612321, 0.5554828317581426]],
    limit=5, # Max. number of search results to return
    search_params={"metric_type": "IP", "params": {}}, # Search parameters
    partition_names=["partition_1"] # Partition names to search in
)

result = json.dumps(res, indent=4)
print(result)
```

输出类似于以下内容：

```python
[
    [
        {
            "id": 16,
            "distance": 0.9200337529182434,
            "entity": {}
        },
        {
            "id": 14,
            "distance": 0.4505271911621094,
            "entity": {}
        },
        {
            "id": 15,
            "distance": 0.19924677908420563,
            "entity": {}
        },
        {
            "id": 17,
            "distance": 0.0075093843042850494,
            "entity": {}
        },
        {
            "id": 13,
            "distance": -0.14609718322753906,
            "entity": {}
        }
    ]
]
```

然后，在`partition_2`中搜索实体：

```python
# Create a MilvusClient instance
client = MilvusClient(
    uri="http://localhost:19530",
)

# Search in partition_2
res = client.search(
    collection_name="test_collection", # Replace with the actual name of your collection
    data=[[-0.2798451532635784, 0.9486592746891414, -0.9311928407781922, 0.1830057032090473, 0.6962886429672028]],
    limit=5, # Max. number of search results to return
    search_params={"metric_type": "IP", "params": {}}, # Search parameters
    partition_names=["partition_2"] # Partition names to search in
)

result = json.dumps(res, indent=4)
print(result)
```

输出类似于以下内容：

```python
[
    [
        {
            "id": 20,
            "distance": 2.363696813583374,
            "entity": {}
        },
        {
            "id": 26,
            "distance": 1.0665391683578491,
            "entity": {}
        },
        {
            "id": 23,
            "distance": 1.066049575805664,
            "entity": {}
        },
        {
            "id": 29,
            "distance": 0.8353596925735474,
            "entity": {}
        },
        {
            "id": 28,
            "distance": 0.7484277486801147,
            "entity": {}
        }
    ]
]
```

`partition_1`中的数据与`partition_2`中的数据不同。因此，搜索结果将被限制到指定的分区，反映该子集的独特特征和数据分布。


### 使用输出字段搜索

使用输出字段搜索允许您指定匹配向量的哪些属性或字段应包含在搜索结果中。

您可以在请求中指定`output_fields`以返回具有特定字段的结果。

下面是返回带有`color`属性值的结果的示例：

In [11]:
# Search with output fields
res = client.search(
    collection_name="quick_setup", # Replace with the actual name of your collection
    data=[[0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592]],
    limit=5, # Max. number of search results to return
    search_params={"metric_type": "IP", "params": {}}, # Search parameters
    output_fields=["color"] # Output fields to return
)

result = json.dumps(res, indent=4)
print(result)

[
    [
        {
            "id": 572,
            "distance": 1.870904803276062,
            "entity": {
                "color": "grey_6603"
            }
        },
        {
            "id": 894,
            "distance": 1.842880129814148,
            "entity": {
                "color": "red_9899"
            }
        },
        {
            "id": 751,
            "distance": 1.6808228492736816,
            "entity": {
                "color": "yellow_7628"
            }
        },
        {
            "id": 673,
            "distance": 1.6371359825134277,
            "entity": {
                "color": "purple_4815"
            }
        },
        {
            "id": 364,
            "distance": 1.6252869367599487,
            "entity": {
                "color": "white_2448"
            }
        }
    ]
]




输出类似于以下内容：

```python
[
    [
        {
            "id": 0,
            "distance": 1.4093276262283325,
            "entity": {
                "color": "pink_8682"
            }
        },
        {
            "id": 16,
            "distance": 1.0159327983856201,
            "entity": {
                "color": "yellow_1496"
            }
        },
        {
            "id": 4,
            "distance": 0.9902134537696838,
            "entity": {
                "color": "red_4794"
            }
        },
        {
            "id": 14,
            "distance": 0.9803846478462219,
            "entity": {
                "color": "green_2899"
            }
        },
        {
            "id": 1,
            "distance": 0.8519943356513977,
            "entity": {
                "color": "red_7025"
            }
        }
    ]
]
```

除了最近的邻居，搜索结果将包括指定的字段`color`，为每个匹配向量提供更丰富的信息集。


## 过滤的搜索

筛选搜索将标量筛选器应用于矢量搜索，允许您根据特定条件优化搜索结果。您可以在[布尔表达式规则](https://milvus.io/docs/boolean.md)中找到有关过滤器表达式的更多信息，并在[获取标量查询](https://milvus.io/docs/get-and-scalar-query.md)中找到示例。

### 使用 `like` 操作者

`like`运算符通过评估包括前缀、中缀和后缀在内的模式来增强字符串搜索：

- 前缀匹配：要查找以特定前缀开头的值，请使用语法`'like "prefix%"'`。
- 中缀匹配：要在字符串中的任何位置查找包含特定字符序列的值，请使用语法`'like "%infix%"'`。
- 后缀匹配：要查找以特定后缀结尾的值，请使用语法`'like "%suffix"'`。

对于单字符匹配，下划线（`_`）充当一个字符的下划线，例如，`'like "y_llow"'`.

### 搜索字符串中的特殊字符

如果要搜索包含特殊字符（如下划线（`_`）或百分比符号（`%`））的字符串，这些字符通常在搜索模式中用作通配符（`_`用于任何单个字符，`%`用于任何字符序列），则必须对这些字符进行转义以将其视为原义字符。使用反斜杠（`\`）来转义特殊字符，并记住转义反斜杠本身。例如：

- 要搜索文本下划线，请使用`\\_`。
- 要搜索文字百分比符号，请使用`\\%`。

因此，如果您需要搜索文本`"_version_"`，您的查询应格式化为`'like "\\_version\\_"'`，以确保下划线被视为搜索词的一部分，而不是通配符。

过滤颜色前缀为红色的结果：

In [12]:
# Search with filter
res = client.search(
    collection_name="quick_setup", # Replace with the actual name of your collection
    data=[[0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592]],
    limit=5, # Max. number of search results to return
    search_params={"metric_type": "IP", "params": {}}, # Search parameters
    output_fields=["color"], # Output fields to return
    filter='color like "red%"'
)

result = json.dumps(res, indent=4)
print(result)

[
    [
        {
            "id": 894,
            "distance": 1.842880129814148,
            "entity": {
                "color": "red_9899"
            }
        },
        {
            "id": 554,
            "distance": 1.577196478843689,
            "entity": {
                "color": "red_8375"
            }
        },
        {
            "id": 395,
            "distance": 1.3502776622772217,
            "entity": {
                "color": "red_2586"
            }
        },
        {
            "id": 190,
            "distance": 1.217180848121643,
            "entity": {
                "color": "red_2064"
            }
        },
        {
            "id": 616,
            "distance": 1.204943060874939,
            "entity": {
                "color": "red_2544"
            }
        }
    ]
]


输出类似于以下内容：

```python
[
    [
        {
            "id": 4,
            "distance": 0.9902134537696838,
            "entity": {
                "color": "red_4794"
            }
        },
        {
            "id": 1,
            "distance": 0.8519943356513977,
            "entity": {
                "color": "red_7025"
            }
        },
        {
            "id": 6,
            "distance": -0.4113418459892273,
            "entity": {
                "color": "red_9392"
            }
        }
    ]
]
```


过滤字符串中颜色包含字母`11`的结果：

In [15]:
# Infix match on color field
res = client.search(
    collection_name="quick_setup", # Replace with the actual name of your collection
    data=[[0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592]],
    limit=5, # Max. number of search results to return
    search_params={"metric_type": "IP", "params": {}}, # Search parameters
    output_fields=["color"], # Output fields to return
    filter='color like "%ll%"' # Filter on color field, infix match on "ll"
)

result = json.dumps(res, indent=4)
print(result)

[
    [
        {
            "id": 751,
            "distance": 1.6808228492736816,
            "entity": {
                "color": "yellow_7628"
            }
        },
        {
            "id": 102,
            "distance": 1.3870998620986938,
            "entity": {
                "color": "yellow_4662"
            }
        },
        {
            "id": 848,
            "distance": 1.2569729089736938,
            "entity": {
                "color": "yellow_5906"
            }
        },
        {
            "id": 411,
            "distance": 1.254452109336853,
            "entity": {
                "color": "yellow_2709"
            }
        },
        {
            "id": 77,
            "distance": 1.1365606784820557,
            "entity": {
                "color": "yellow_1366"
            }
        }
    ]
]




输出类似于以下内容：

```python
[
    [
        {
            "id": 5,
            "distance": 0.7972343564033508,
            "entity": {
                "color": "yellow_4222"
            }
        }
    ]
]
```

## 范围搜索

范围搜索允许您查找位于查询向量的指定距离范围内的向量。

通过设置`radius`和可选的`range_filter`，您可以调整搜索的广度，以包括与查询向量有些相似的向量，从而提供对潜在匹配的更全面的视图。

- `radius`：定义搜索空间的外部边界。只有与查询向量在此距离内的向量才被视为潜在匹配。
- `range_filter`：虽然`radius`设置搜索的外部限制，但`range_filter`可以可选地用于定义内部边界，创建一个距离范围，在该范围内矢量必须被视为匹配。


In [22]:
# Conduct a range search
search_params = {
    "metric_type": "IP",
    "params": {
        "radius": 0.8, # Radius of the search circle
        "range_filter": 1.0 # Range filter to filter out vectors that are not within the search circle
    }
}

res = client.search(
    collection_name="quick_setup", # Replace with the actual name of your collection
    data=[[0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592]],
    limit=3, # Max. number of search results to return
    search_params=search_params, # Search parameters
    output_fields=["color"], # Output fields to return
)

result = json.dumps(res, indent=4)
print(result)

[
    [
        {
            "id": 697,
            "distance": 0.9987217783927917,
            "entity": {
                "color": "orange_9374"
            }
        },
        {
            "id": 415,
            "distance": 0.9968745708465576,
            "entity": {
                "color": "purple_2488"
            }
        },
        {
            "id": 371,
            "distance": 0.9961320161819458,
            "entity": {
                "color": "blue_5769"
            }
        }
    ]
]


输出类似于以下内容：

```python
[
    [
        {
            "id": 4,
            "distance": 0.9902134537696838,
            "entity": {
                "color": "red_4794"
            }
        },
        {
            "id": 14,
            "distance": 0.9803846478462219,
            "entity": {
                "color": "green_2899"
            }
        },
        {
            "id": 1,
            "distance": 0.8519943356513977,
            "entity": {
                "color": "red_7025"
            }
        }
    ]
]
```

您将观察到，返回的所有实体与查询向量的距离都在福尔斯的0.8到1.0之间。

`radius`和`range_filter`的参数设置随使用的度量类型而异。

| **度量标准类型** | **释义**                   | **范围搜索设置**                                             |
| :--------------- | :----------------------------- | :----------------------------------------------------------- |
| `L2`            | 较小的L2距离表示较高的相似性。 | 要从结果中排除最接近的向量，请确保： `range_filter`<=距离<`radius` |
| `IP`            | IP距离越大，相似性越高。       | 要从结果中排除最接近的向量，请确保： `radius`<距离<= `range_filter` |


## 分组检索

在Milvus中，按特定字段分组搜索可以避免结果中相同字段项的冗余。您可以为特定字段获得一组不同的结果。

考虑一个文档集合，每个文档分成不同的段落。每个段落由一个向量嵌入表示，并且属于一个文档。要查找相关文档而不是相似的段落，您可以在`group_by_field`操作中包含`search()`参数，以按文档ID对结果进行分组。这有助于返回最相关和唯一的文档，而不是来自同一文档的单独段落。

下面是按字段对搜索结果进行分组的示例代码：

In [35]:
# Connect to Milvus
client = MilvusClient(uri='http://localhost:19530') # Milvus server address

# Load data into collection
client.load_collection("quick_setup") # Collection name

# Group search results
res = client.search(
    collection_name="quick_setup", # Collection name
    data=[[0.14529211512077012, 0.9147257273453546, 0.7965055218724449, 0.7009258593102812, 0.5605206522382088]], # Query vector
    search_params={
        "metric_type": "IP",
        "params": {"nprobe": 10},
        }, # Search parameters
    limit=2, # Max. number of search results to return
    group_by_field="id", # Group results by document ID
    output_fields=["color",]
)

result = json.dumps(res, indent=4)
print(result)

# # Retrieve the values in the `doc_id` column
# doc_ids = [result['entity']['doc_id'] for result in res[0]]

# print(doc_ids)

[
    [
        {
            "id": 48,
            "distance": 2.7333407402038574,
            "entity": {
                "color": "purple_5797"
            }
        },
        {
            "id": 94,
            "distance": 2.532440662384033,
            "entity": {
                "color": "yellow_3517"
            }
        }
    ]
]


输出类似于以下内容：

```python
[5, 10, 1, 7, 9, 6, 3, 4, 8, 2]
```

在给定的输出中，可以观察到返回的实体不包含任何重复的`doc_id`值。

为了进行比较，让我们注释掉`group_by_field`并进行常规搜索：

```python
# Connect to Milvus
client = MilvusClient(uri='http://localhost:19530') # Milvus server address

# Load data into collection
client.load_collection("group_search") # Collection name

# Search without `group_by_field`
res = client.search(
    collection_name="group_search", # Collection name
    data=query_passage_vector, # Replace with your query vector
    search_params={
    "metric_type": "L2",
    "params": {"nprobe": 10},
    }, # Search parameters
    limit=10, # Max. number of search results to return
    # group_by_field="doc_id", # Group results by document ID
    output_fields=["doc_id", "passage_id"]
)

# Retrieve the values in the `doc_id` column
doc_ids = [result['entity']['doc_id'] for result in res[0]]

print(doc_ids)
```

输出类似于以下内容：

```python
[1, 10, 3, 10, 1, 9, 4, 4, 8, 6]
```

在给定的输出中，可以观察到返回的实体包含重复的`doc_id`值。

**限制**

- 索引：此分组功能仅适用于使用HNSW、IVF_FLAT或FLAT类型索引的集合。有关详细信息，请参阅内存索引。
- Vector：目前，分组搜索不支持BINARY_VECTOR类型的向量字段。有关数据类型的详细信息，请参阅[支持的数据类型](https://milvus.io/docs/schema.md#Supported-data-types)。
- 字段：目前，分组搜索只允许单个列。您不能在`group_by_field`配置中指定多个字段名称。 此外，分组搜索与JSON、FLOAT、DOUBLE、ARRAY或向量字段的数据类型不兼容。
- 性能影响：请注意，性能会随着查询向量计数的增加而降低。使用具有2个CPU核心和8 GB内存的集群作为示例，分组搜索的执行时间随着输入查询向量的数量成比例地增加。
- 功能：目前，[范围搜索](https://milvus.io/docs/single-vector-search.md#Range-search)、[搜索迭代](https://milvus.io/docs/with-iterators.md#Search-with-iterator)器或[多向量搜索](https://milvus.io/docs/multi-vector-search.md)不支持分组搜索


## 搜索参数

在上述搜索中，除范围搜索外，默认搜索参数适用。在正常情况下，您不需要手动设置搜索参数。


```python
# In normal cases, you do not need to set search parameters manually
# Except for range searches.
search_parameters = {
    'metric_type': 'L2',
    'params': {
        'nprobe': 10,
        'level': 1，
        'radius': 1.0
        'range_filter': 0.8
    }
}
```

下表列出了搜索参数中所有可能的设置。

| **参数名称** | **参数描述**                                                 |
| :----------- | :----------------------------------------------------------- |
| `metric_type`        | 如何度量向量嵌入之间的相似性。 可能的值为`IP`、`L2`和`COSINE`，默认值为加载的索引文件的值。 |
| `params.nprobe`        | 搜索期间要查询的单位数。 值福尔斯落在范围[1，nlist [1]]内。  |
| `params.level`        | 搜索精度级别。 可能的值为`1`、`2`和`3`，默认值为`1`。值越大，结果越准确，但性能越差。 |
| `params.radius`        | 查询向量和候选向量之间的最小相似度。 值福尔斯落在范围[1，nlist [1]]内。 |
| `params.range_filter`        | 相似性范围，可选地细化对落在该范围内的向量的搜索。 值福尔斯落在范围[top-K [2]，∞]中。 |

**注意到**

[1]索引后的群集单元数。当索引集合时，Milvus将矢量数据细分为多个聚类单元，其数量随实际索引设置而变化。

[2]搜索中要返回的实体数。

------

# 多向量搜索

自Milvus 2.4以来，我们引入了多向量支持和混合搜索框架，这意味着用户可以将多个向量场（最多10个）带入单个集合。不同的向量场可以代表不同的方面，不同的嵌入模型，甚至是表征同一实体的数据的不同模态，这极大地扩展了信息的丰富性。此功能在综合搜索场景中特别有用，例如基于图片，语音，指纹等各种属性在矢量库中识别最相似的人。

多向量搜索允许在各种向量字段上执行搜索请求，并使用重新排序策略（例如倒数秩融合（RRF）和加权评分）组合结果。要了解有关重新排序策略的更多信息，请参阅[重新排序](https://milvus.io/docs/reranking.md)。

在本教程中，您将学习如何：

- 创建多个`AnnSearchRequest`实例，用于在不同向量场上进行相似性搜索;
- 配置重排序策略，对多个`AnnSearchRequest`实例的搜索结果进行联合收割机合并重排序;
- 使用`hybrid_search()`方法执行多向量搜索。

此页面上的代码片段使用PyMilvus ORM模块与Milvus交互。新的MilvusClient SDK的代码片段将很快提供。


## 筹备工作

在开始多向量搜索之前，请确保您有一个包含多个向量字段的集合。

下面是创建一个名为`test_collection`的集合的示例，其中包含两个向量字段`filmVector`和`posterVector`，并将随机实体插入其中。


In [36]:
from pymilvus import connections, Collection, FieldSchema, CollectionSchema, DataType
import random

# 1. Set up a Milvus client
client = MilvusClient(
    uri="http://localhost:19530"
)

if client.has_collection("test_collection"):
    client.drop_collection("test_collection")

In [37]:
# Connect to Milvus
connections.connect(
    host="localhost", # Replace with your Milvus server IP
    port="19530"
)

# Create schema
fields = [
    FieldSchema(name="film_id", dtype=DataType.INT64, is_primary=True),
    FieldSchema(name="filmVector", dtype=DataType.FLOAT_VECTOR, dim=5), # Vector field for film vectors
    FieldSchema(name="posterVector", dtype=DataType.FLOAT_VECTOR, dim=5)] # Vector field for poster vectors

schema = CollectionSchema(fields=fields,enable_dynamic_field=False)

# Create collection
collection = Collection(name="test_collection", schema=schema)

In [38]:
# Create index for each vector field
index_params = {
    "metric_type": "L2",
    "index_type": "IVF_FLAT",
    "params": {"nlist": 128},
}

collection.create_index("filmVector", index_params)
collection.create_index("posterVector", index_params)

# Generate random entities to insert
entities = []

for _ in range(1000):
    # generate random values for each field in the schema
    film_id = random.randint(1, 1000)
    film_vector = [ random.random() for _ in range(5) ]
    poster_vector = [ random.random() for _ in range(5) ]

    # creat a dictionary for each entity
    entity = {
        "film_id": film_id,
        "filmVector": film_vector,
        "posterVector": poster_vector
    }

    # add the entity to the list
    entities.append(entity)
    
collection.insert(entities)

(insert count: 1000, delete count: 0, upsert count: 0, timestamp: 449798074724777988, success count: 1000, err count: 0)

## 步骤1：创建多个AnnSearchRequest

多向量搜索使用`hybrid_search()`API在单个调用中执行多个ANN搜索请求。每个`AnnSearchRequest`表示对特定向量字段的单个搜索请求。

下面的示例创建两个`AnnSearchRequest`实例，以在两个向量场上执行单独的相似性搜索。

In [39]:
from pymilvus import AnnSearchRequest

# Create ANN search request 1 for filmVector
query_filmVector = [[0.8896863042430693, 0.370613100114602, 0.23779315077113428, 0.38227915951132996, 0.5997064603128835]]


In [40]:
search_param_1 = {
    "data": query_filmVector, # Query vector
    "anns_field": "filmVector", # Vector field name
    "param": {
        "metric_type": "L2", # This parameter value must be identical to the one used in the collection schema
        "params": {"nprobe": 10}
    },
    "limit": 2 # Number of search results to return in this AnnSearchRequest
}
request_1 = AnnSearchRequest(**search_param_1)

In [41]:
# Create ANN search request 2 for posterVector
query_posterVector = [[0.02550758562349764, 0.006085637357292062, 0.5325251250159071, 0.7676432650114147, 0.5521074424751443]]
search_param_2 = {
    "data": query_posterVector, # Query vector
    "anns_field": "posterVector", # Vector field name
    "param": {
        "metric_type": "L2", # This parameter value must be identical to the one used in the collection schema
        "params": {"nprobe": 10}
    },
    "limit": 2 # Number of search results to return in this AnnSearchRequest
}
request_2 = AnnSearchRequest(**search_param_2)

# Store these two requests as a list in `reqs`
reqs = [request_1, request_2]

In [42]:
reqs


产品参数：

- `AnnSearchRequest`（对象）

  表示ANN搜索请求的类。每个混合搜索一次可以包含1到1，024个`ANNSearchRequest`对象。

- `data`（列表）

  要在单个`AnnSearchRequest`中搜索的查询向量。目前，此参数接受仅包含单个查询向量的列表，例如`[[0.5791814851218929, 0.5792985702614121, 0.8480776460143558, 0.16098005945243, 0.2842979317256803]]`。将来，此参数将扩展为接受多个查询向量。

- `anns_field`（字符串）

  要在单个`AnnSearchRequest`中使用的向量字段的名称。

- `param`（dict）

  单个`AnnSearchRequest`的搜索参数字典。这些搜索参数与单向量搜索的搜索参数相同。有关详细信息，请参阅[搜索参数](https://milvus.io/docs/single-vector-search.md#Search-parameters)。

- `limit`（int）

  要包含在单个`ANNSearchRequest`中的最大搜索结果数。

  此参数仅影响单个`ANNSearchRequest`中返回的搜索结果的数量，而不决定`hybrid_search`调用返回的最终结果。在混合搜索中，最终结果是通过组合和重新排序来自多个`ANNSearchRequest`实例的结果来确定的。


## 步骤2：配置重新排名策略

创建`AnnSearchRequest`实例后，配置重新排序策略，对结果进行联合收割机合并和重新排序。目前有两个选项：`WeightedRanker`和`RRFRanker`。有关重新排序策略的更多信息，请参阅[重新排序](https://milvus.io/docs/reranking.md)。

- 使用加权评分

  `WeightedRanker`用于为具有指定权重的每个向量场搜索的结果分配重要性。如果您将某些向量场的优先级设置为高于其他向量场，`WeightedRanker(value1, value2, ..., valueN)`可以在组合搜索结果中反映这一点。


In [43]:
from pymilvus import WeightedRanker
# Use WeightedRanker to combine results with specified weights
# Assign weights of 0.8 to text search and 0.2 to image search
rerank = WeightedRanker(0.8, 0.2)  



  使用`WeightedRanker`时，请注意：

  - 每个权重值的范围从0（最不重要）到1（最重要），影响最终的汇总分数。
  - `WeightedRanker`中提供的权重值总数应等于您创建的`AnnSearchRequest`实例数。

- 使用倒数等级融合（RFF）

  ```python
  # Alternatively, use RRFRanker for reciprocal rank fusion reranking
  from pymilvus import RRFRanker
  
  rerank = RRFRanker()
  ```

## 步骤3：执行混合搜索

设置了`AnnSearchRequest`实例和重排序策略后，使用`hybrid_search()`方法执行多向量搜索。


In [44]:
# Before conducting multi-vector search, load the collection into memory.
collection.load()

res = collection.hybrid_search(
    reqs, # List of AnnSearchRequests created in step 1
    rerank, # Reranking strategy specified in step 2
    limit=2 # Number of final search results to return
)

print(res)

["['id: 250, distance: 0.7905547022819519, entity: {}', 'id: 226, distance: 0.7776205539703369, entity: {}']"]


产品参数：

- `data`（列表）

  一个搜索请求列表，其中每个请求都是一个`ANNSearchRequest`对象。每个请求可以对应于不同的向量场和不同的搜索参数集合。

- `AnnSearchRequest`（对象）

  用于混合搜索的重排序策略。可能的值：`WeightedRanker(value1, value2, ..., valueN)`和`RRFRanker()`。

  有关重新排序策略的更多信息，请参阅[重新排序](https://milvus.io/docs/reranking.md)。

- `limit`（int）

  在混合搜索中返回的最大最终结果数。

输出类似于以下内容：

```python
["['id: 844, distance: 0.006047376897186041, entity: {}', 'id: 876, distance: 0.006422005593776703, entity: {}']"]
```

## 限制

- 通常，每个集合默认允许最多4个向量字段。但是，您可以选择调整`proxy.maxVectorFieldNum`配置以扩展集合中的最大向量字段数，每个集合的最大限制为10个向量字段。更多信息请参见[代理相关插件](https://milvus.io/docs/configure_proxy.md#Proxy-related-Configurations)。
- 集合中的部分索引或加载向量字段将导致错误。
- 目前，混合搜索中的每个`AnnSearchRequest`只能携带一个查询向量。

## FAQ

- **在哪种情况下推荐使用多向量搜索？**

  多向量搜索是要求高精度的复杂情况的理想选择，特别是当一个实体可以由多个不同的向量表示时。这适用于通过不同的嵌入模型处理相同数据（如句子）或将多模态信息（如个人的图像，指纹和声纹）转换为各种矢量格式的情况。通过给这些向量分配权重，它们的组合影响可以显着丰富召回率并提高搜索结果的有效性。

- **加权排序器如何将不同向量场之间的距离归一化？**

  加权排序器使用分配给每个字段的权重来规范化向量字段之间的距离。它根据权重计算每个向量场的重要性，优先考虑权重较高的向量场。建议在ANN搜索请求中使用相同的度量类型，以确保一致性。这种方法确保了被认为更重要的向量对整体排名有更大的影响。

- **是否可以使用其他排名器，如Cohere Ranker或BGE Ranker？**

  目前，仅支持提供的排名。计划包括更多的排名正在进行中，为未来的更新。

- **是否可以同时进行多个混合搜索操作？**

  支持，支持同时执行多个混合搜索操作。

- **我可以在多个AnnSearchRequest对象中使用相同的向量字段来执行混合搜索吗？**

  从技术上讲，可以在多个AnnSearchRequest对象中使用相同的向量字段进行混合搜索。混合搜索不需要多个向量场。

------

# Get & Scalar Query获取标量查询

本指南演示如何通过ID获取实体并进行标量过滤。标量筛选检索与指定筛选条件匹配的实体。

## 概述

标量查询基于使用布尔表达式定义的条件筛选集合中的实体。查询结果是一组与定义的条件匹配的实体。与向量搜索不同，它识别集合中与给定向量最接近的向量，查询基于特定标准过滤实体。

在Milvus中，过滤器总是一个字符串，它压缩了由运算符连接的字段名称。在本指南中，您将找到各种过滤器示例。要了解有关操作符详细信息的更多信息，请转到[参考](https://milvus.io/docs/get-and-scalar-query.md#Reference-on-scalar-filters)部分。

此页面上的代码片段使用新[的MilvusClient](https://milvus.io/api-reference/pymilvus/v2.4.x/About.md)（Python）与Milvus交互。其他语言的新MilvusClient SDK将在未来的更新中发布。



## 筹备工作

以下步骤重新调整代码的用途，以连接到Milvus，快速设置集合，并将1，000多个随机生成的实体插入到集合中。


### 步骤1：创建 Collection

In [47]:
from pymilvus import MilvusClient

# 1. Set up a Milvus client
client = MilvusClient(
    uri="http://localhost:19530"
)

if client.has_collection("quick_setup"):
    client.drop_collection("quick_setup")

# 2. Create a collection
client.create_collection(
    collection_name="quick_setup",
    dimension=5,
)

### 步骤2：插入随机生成的实体

In [48]:
# 3. Insert randomly generated vectors 
colors = ["green", "blue", "yellow", "red", "black", "white", "purple", "pink", "orange", "brown", "grey"]
data = [ {
        "id": i, 
        "vector": [ random.uniform(-1, 1) for _ in range(5) ], 
        "color": random.choice(colors), 
        "tag": random.randint(1000, 9999) 
    } for i in range(1000) ]

for i in data:
    i["color_tag"] = "{}_{}".format(i["color"], i["tag"])

print(data[0])


{'id': 0, 'vector': [0.09511546909433033, -0.9058442894044367, -0.45423166762539924, -0.17148939187609447, 0.8446262767540593], 'color': 'blue', 'tag': 7010, 'color_tag': 'blue_7010'}


In [50]:
# 4. Insert entities to the collection
res = client.insert(
    collection_name="quick_setup",
    data=data
)

# print(res)

### 步骤3：创建分区并插入更多实体


In [53]:
# 5. Create two partitions
client.create_partition(collection_name="quick_setup", partition_name="partitionA")
client.create_partition(collection_name="quick_setup", partition_name="partitionB")

# 6. Insert 500 entities in partition A
data = [ {
        "id": i + 1000, 
        "vector": [ random.uniform(-1, 1) for _ in range(5) ], 
        "color": random.choice(colors), 
        "tag": random.randint(1000, 9999) 
    } for i in range(500) ]

for i in data:
    i["color_tag"] = "{}_{}".format(i["color"], i["tag"])

res = client.insert(
    collection_name="quick_setup",
    data=data,
    partition_name="partitionA"
)

# print(res)

In [54]:
# 7. Insert 300 entities in partition B
data = [ {
        "id": i + 1500, 
        "vector": [ random.uniform(-1, 1) for _ in range(5) ], 
        "color": random.choice(colors), 
        "tag": random.randint(1000, 9999) 
    } for i in range(300) ]

for i in data:
    i["color_tag"] = "{}_{}".format(i["color"], i["tag"])

res = client.insert(
    collection_name="quick_setup",
    data=data,
    partition_name="partitionB"
)

# print(res)

## 按ID获取实体

如果你知道你感兴趣的实体的ID，你可以使用`get()`方法。

In [55]:
# 4. Get entities by ID
res = client.get(
    collection_name="quick_setup",
    ids=[0, 1, 2]
)

print(res)

[{'color': 'blue', 'tag': 7010, 'color_tag': 'blue_7010', 'id': 0, 'vector': [0.09511547, -0.9058443, -0.45423168, -0.17148939, 0.84462625]}, {'color': 'yellow', 'tag': 6317, 'color_tag': 'yellow_6317', 'id': 1, 'vector': [0.56778526, -0.19224258, -0.5964742, -0.48585027, -0.6893527]}, {'color': 'pink', 'tag': 4596, 'color_tag': 'pink_4596', 'id': 2, 'vector': [0.4412155, -0.39091882, 0.77029777, 0.8460942, -0.03598112]}]


### 从分区获取实体

您还可以从特定分区获取实体。

In [56]:
# 5. Get entities from partitions
res = client.get(
    collection_name="quick_setup",
    ids=[0, 1, 2],
    partition_names=["_default"]
)

print(res)

[{'color': 'blue', 'tag': 7010, 'color_tag': 'blue_7010', 'id': 0, 'vector': [0.09511547, -0.9058443, -0.45423168, -0.17148939, 0.84462625]}, {'color': 'yellow', 'tag': 6317, 'color_tag': 'yellow_6317', 'id': 1, 'vector': [0.56778526, -0.19224258, -0.5964742, -0.48585027, -0.6893527]}, {'color': 'pink', 'tag': 4596, 'color_tag': 'pink_4596', 'id': 2, 'vector': [0.4412155, -0.39091882, 0.77029777, 0.8460942, -0.03598112]}]


## 使用基本运算符

在本节中，您将找到如何在标量过滤中使用基本运算符的示例。您也可以将这些过滤器应用于[矢量搜索](https://milvus.io/docs/single-vector-search.md#Filtered-search)和[数据删除](https://milvus.io/docs/insert-update-delete.md#Delete-entities)。

- 过滤标记值介于1，000到1，500之间的图元。


In [57]:
res = client.query(
  collection_name="quick_setup",
  # highlight-start
  filter="1000 < tag < 1500",
  output_fields=["color_tag"],
  # highlight-end
  limit=3
)
res

[{'color_tag': 'green_1024', 'id': 29}, {'color_tag': 'blue_1272', 'id': 32}]

- 过滤实体，并将其颜色值设置为红色。

In [58]:
res = client.query(
      collection_name="quick_setup",
      # highlight-start
      filter='color == "brown"',
      output_fields=["color_tag"],
      # highlight-end
      limit=3
  )
res

[{'color_tag': 'brown_7123', 'id': 10}, {'color_tag': 'brown_3260', 'id': 14}]

- 过滤颜色值未设置为绿色和紫色的图元。


In [59]:
res = client.query(
  collection_name="quick_setup",
  # highlight-start
  filter='color not in ["green", "purple"]',
  output_fields=["color_tag"],
  # highlight-end
  limit=3
)
res

[{'color_tag': 'blue_7010', 'id': 0}, {'color_tag': 'yellow_6317', 'id': 1}]

- 过滤颜色标签以红色开头的文章。

In [60]:
res = client.query(
    collection_name="quick_setup",
    # highlight-start
    filter='color_tag like "red%"',
    output_fields=["color_tag"],
    # highlight-end
    limit=3
)
res

[{'color_tag': 'red_8538', 'id': 12}, {'color_tag': 'red_4840', 'id': 31}]

- 过滤颜色设置为红色且标记值在1，000到1，500范围内的图元。

In [61]:
res = client.query(
      collection_name="quick_setup",
      # highlight-start
      filter='(color == "red") and (1000 < tag < 1500)',
      output_fields=["color_tag"],
      # highlight-end
      limit=3
  )
res

[{'id': 188, 'color_tag': 'red_1426'}, {'id': 264, 'color_tag': 'red_1412'}]

## 使用高级运算符

在本节中，您将找到如何在标量过滤中使用高级运算符的示例。您也可以将这些过滤器应用于矢量搜索和数据删除。

### 计数实体

- 计算集合中实体的总数。


In [62]:
res = client.query(
  collection_name="quick_setup",
  # highlight-start
  output_fields=["count(*)"]
  # highlight-end
)
res

[{'count(*)': 3800}]

- 计算特定分区中的实体总数。


In [63]:
res = client.query(
      collection_name="quick_setup",
      # highlight-start
      output_fields=["count(*)"],
      partition_name="partitionA"
      # highlight-end
  )
res

[{'count(*)': 3800}]

In [64]:
res = client.query(
      collection_name="quick_setup",
      # highlight-start
      output_fields=["count(*)"],
      partition_name="partitionB"
      # highlight-end
  )
res

[{'count(*)': 3800}]

- 统计与筛选条件匹配的实体数

In [65]:
res = client.query(
      collection_name="quick_setup",
      # highlight-start
      filter='(publication == "Towards Data Science") and ((claps > 1500 and responses > 15) or (10 < reading_time < 15))',
      output_fields=["count(*)"],
      # highlight-end
  )
res

[{'count(*)': 0}]



## 标量滤波器参考

### 基础运营商

布尔表达式始终是由操作符连接的字段名称组成的字符串。在本节中，您将了解有关基本运算符的更多信息。

| **操作者** | **描述**                                                     |
| :--------- | :----------------------------------------------------------- |
| add (&&)   | 如果两个操作数都为true，则为True                             |
| or (\|\|)  | 如果任一操作数为true，则返回True                             |
| +，-，*，/ | 加减乘除                                                     |
| ***\***    | 指数                                                         |
| **%**      | 模量                                                         |
| <、>       | 小于，大于                                                   |
| ==，！=    | 等于，不等于                                                 |
| <=，>=     | 小于等于，大于等于                                           |
| not         | 反转给定条件的结果。                                         |
| like        | 将一个值与使用通配符的类似值进行比较。 例如，“prefix%”匹配以“prefix”开始的字符串。 |
| in         | 测试表达式是否与值列表中的任何值匹配。                       |

### 高级操作

- `count(*)`

  计算集合中实体的确切数目。将此字段用作输出字段，以获取集合或分区中实体的确切数量。

  **注意到**

  这适用于加载的集合。您应该将其用作唯一的输出字段。

------

# 关于Iterator

Milvus提供了搜索和查询迭代器，用于迭代具有大量实体的结果。


## 概述

迭代器是功能强大的工具，可帮助您使用主键值和布尔表达式在大型数据集中导航。这可以显著改善您检索数据的方式。与偏移量和限制参数的传统使用不同，迭代器提供了一种更具可伸缩性的解决方案，这些参数可能会随着时间的推移而变得效率低下。


### 使用迭代器的好处

- 简单：消除复杂的偏移和限制设置。
- 效率：通过只获取需要的数据来提供可伸缩的数据检索。
- 一致性：使用布尔过滤器确保数据集大小一致。

**注意到**

- 此功能适用于Milvus 2.3.x或更高版本。
- 此页面上的代码片段使用新[的MilvusClient](https://milvus.io/api-reference/pymilvus/v2.4.x/About.md)（Python）与Milvus交互。其他语言的新MilvusClient SDK将在未来的更新中发布。


## 筹备工作

以下步骤重新调整代码的用途，以连接到Milvus，快速设置集合，并将超过10，000个随机生成的实体插入到集合中。

### 步骤1：创建收藏

In [66]:
from pymilvus import MilvusClient

# 1. Set up a Milvus client
client = MilvusClient(
    uri="http://localhost:19530"
)

if client.has_collection("quick_setup"):
    client.drop_collection("quick_setup")

# 2. Create a collection
client.create_collection(
    collection_name="quick_setup",
    dimension=5,
)

### 步骤2：插入随机生成的实体

In [67]:
# 3. Insert randomly generated vectors 
colors = ["green", "blue", "yellow", "red", "black", "white", "purple", "pink", "orange", "brown", "grey"]
data = [ {
        "id": i, 
        "vector": [ random.uniform(-1, 1) for _ in range(5) ], 
        "color": random.choice(colors), 
        "tag": random.randint(1000, 9999) 
    } for i in range(10000) ]

for i in data:
    i["color_tag"] = "{}_{}".format(i["color"], i["tag"])

print(data[0])

{'id': 0, 'vector': [0.7454548690387337, -0.6534701931393274, 0.8867624550293634, 0.9867488810367115, -0.7136163936475912], 'color': 'pink', 'tag': 3096, 'color_tag': 'pink_3096'}


In [69]:
# 4. Insert entities to the collection
res = client.insert(
    collection_name="quick_setup",
    data=data
)

# print(res)

## 使用迭代器搜索

迭代器使相似性搜索更具可扩展性。要使用迭代器进行搜索，请执行以下操作：

1. 初始化搜索迭代器以定义搜索参数和输出字段。
2. 在循环中使用next（）方法对搜索结果进行分页。
   - 如果该方法返回空数组，则循环结束，不再有可用页。
   - 所有结果都带有指定的输出字段。
3. 在检索完所有数据后，手动调用close（）方法关闭迭代器。

In [71]:
from pymilvus import Collection

# 1. Get the prepared collection
collection = Collection("quick_setup")

# 2. Set up the search parameters
search_params = {
    "metric_type": "COSINE",
    "params": {}
}

# 3. Initialize a search iterator
iterator = collection.search_iterator(
    data=[[0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592]],
    anns_field="vector",
    batch_size=10, # Controls the size of the return each time you call next()
    param=search_params,
    output_fields=["color_tag"]
)

# 4. Iterate the search results
results = []

In [72]:
while True:
    result = iterator.next()
    if len(result) == 0:
        iterator.close()
        break;
        
    results.extend(result)
    
# 5. Check the search results
print(len(results))

print(results[:3])

10000
[id: 1385, distance: 0.9786712527275085, entity: {'color_tag': 'green_1704'}, id: 7508, distance: 0.966649055480957, entity: {'color_tag': 'yellow_4739'}, id: 9293, distance: 0.964911937713623, entity: {'color_tag': 'orange_3963'}]


## 使用迭代器查询

In [74]:
# 6. Initialize a query iterator
iterator = collection.query_iterator(
    batch_size=10, # Controls the size of the return each time you call next()
    expr="color_tag like \"brown_8\"",
    output_fields=["color_tag"]
)

# 7. Iterator the query results
results = []

while True:
    result = iterator.next()
    if len(result) == 0:
        iterator.close()
        break;
        
    results.extend(result)
    
# 8. Check the search results
print(len(results))

print(results[:3])

0
[]


------